<a href="https://colab.research.google.com/github/elkinkazan/itis-2025-ai-tools/blob/main/tasks/Task1_PyTorch_Iliasova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
!pip install --upgrade gspread pandas google-auth
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import random
# Authenticate and create the PyDrive client.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)



In [1]:
# FILL THIS
student_name = "ELVIRA ILIASOVA"
group_id = "11-451"


In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Kfxj2eDFl7xQnXw7Fpb9bwghc65o8xf--VpNxrdWHaY/edit?gid=0#gid=0"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1

# Ensure header row exists
if not worksheet.get_all_values():
    worksheet.append_row(["Student Name", "Group","TaskID", "Score"])


In [2]:
# MAIN NOTEBOOK GOES HERE
task_id = "Task1_PyTorch"
score = 0
max_score = 15

In [3]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [4]:
# Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)


100%|██████████| 26.4M/26.4M [00:02<00:00, 10.5MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 167kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 3.10MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 10.9MB/s]


In [5]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


# Modelling

In [6]:
# mps for Mac
global device
device = "cuda"
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cuda device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To train a model, we need a loss function and an optimizer.

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return  100*correct, test_loss

In [10]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Epoch 1
-------------------------------
loss: 2.310881  [   64/60000]
loss: 2.294919  [ 6464/60000]
loss: 2.278117  [12864/60000]
loss: 2.273933  [19264/60000]
loss: 2.251389  [25664/60000]
loss: 2.232425  [32064/60000]
loss: 2.232980  [38464/60000]
loss: 2.205950  [44864/60000]
loss: 2.200221  [51264/60000]
loss: 2.164249  [57664/60000]
Test Error: 
 Accuracy: 43.3%, Avg loss: 2.158630 

Epoch 2
-------------------------------
loss: 2.169852  [   64/60000]
loss: 2.155838  [ 6464/60000]
loss: 2.100337  [12864/60000]
loss: 2.122242  [19264/60000]
loss: 2.065565  [25664/60000]
loss: 2.016723  [32064/60000]
loss: 2.039900  [38464/60000]
loss: 1.966455  [44864/60000]
loss: 1.967462  [51264/60000]
loss: 1.897004  [57664/60000]
Test Error: 
 Accuracy: 57.1%, Avg loss: 1.893588 

Epoch 3
-------------------------------
loss: 1.921637  [   64/60000]
loss: 1.889003  [ 6464/60000]
loss: 1.774709  [12864/60000]
loss: 1.823767  [19264/60000]
loss: 1.712031  [25664/60000]
loss: 1.666987  [32064/600

# Task 1
Build graphic for the learning process.

1. Load/Save models

In [17]:
def createModel():
    model = NeuralNetwork().to(device)
    model.load_state_dict(torch.load("model.pth", map_location=device))  # загружаем обученные веса
    model.eval()
    return model

In [22]:
def check_classes(model, test_data, device):
    classes = [
        "T-shirt/top", "Trouser", "Pullover", "Dress", "Coat",
        "Sandal", "Shirt", "Sneaker", "Bag", "Ankle boot",
    ]

    model.eval()
    image, label = test_data[0]                     # Загружаем первый пример
    image = image.unsqueeze(0).to(device)           # Добавляем batch и переносим на device

    with torch.no_grad():
        output = model(image)                       # Предсказание
        predicted_class = output.argmax(1).item()   # Индекс наибольшего логита

    predicted = classes[predicted_class]
    actual = classes[label]

    return predicted, actual

In [23]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_creation(self):
        # Загрузка изображения
        model = createModel()
        predicted, actual = check_classes(model)
        self.assertEqual(predicted, actual, "Prediction wrong")
        global score
        score=score+5

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

E
ERROR: test_model_creation (__main__.TestImageFilters.test_model_creation)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "<ipython-input-23-37fd2b926edf>", line 10, in test_model_creation
    predicted, actual = check_classes(model)
                        ^^^^^^^^^^^^^^^^^^^^
TypeError: check_classes() missing 2 required positional arguments: 'test_data' and 'device'

----------------------------------------------------------------------
Ran 1 test in 0.011s

FAILED (errors=1)


Student Score: 5/15


In [24]:
model = createModel()
predicted, actual = check_classes(model, test_data, device)
print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"


# Task 2. Improve training

In [25]:
import torch.optim as optim

# Improve Model Function
def improveModel():
    epochs = 10
    global device

    optimizer = optim.Adam(model.parameters(), lr=0.001)

    for t in range(epochs):
        print(f"Epoch {t+1}\n-------------------------------")
        train(train_dataloader, model, loss_fn, optimizer)

        acc, loss = test(test_dataloader, model, loss_fn)

        print(f"Test Accuracy after Epoch {t+1}: {acc:.2f}%")
        print(f"Test Loss after Epoch {t+1}: {loss:.4f}")

    return acc, loss

In [ ]:
import unittest
import cv2
import numpy as np

class TestImageFilters(unittest.TestCase):

    def test_model_improve(self):
        acc, loss = improveModel()
        self.assertGreaterEqual(acc, 80, "Model is not accurate")
        global score
        score=score+10

# Run unit tests
if __name__ == "__main__":
    unittest.main(argv=['first-arg-is-ignored'], exit=False)
    print(f"Student Score: {score}/{max_score}")

Epoch 1
-------------------------------
loss: 0.785627  [   64/60000]
loss: 0.516328  [ 6464/60000]
loss: 0.365150  [12864/60000]
loss: 0.500123  [19264/60000]
loss: 0.448951  [25664/60000]
loss: 0.412414  [32064/60000]
loss: 0.361075  [38464/60000]
loss: 0.532576  [44864/60000]
loss: 0.493325  [51264/60000]
loss: 0.520258  [57664/60000]
Test Error: 
 Accuracy: 84.4%, Avg loss: 0.419378 

Test Accuracy after Epoch 1: 84.36%
Test Loss after Epoch 1: 0.4194
Epoch 2
-------------------------------
loss: 0.243778  [   64/60000]
loss: 0.354420  [ 6464/60000]
loss: 0.258789  [12864/60000]
loss: 0.383230  [19264/60000]
loss: 0.441310  [25664/60000]
loss: 0.357029  [32064/60000]
loss: 0.327813  [38464/60000]
loss: 0.516014  [44864/60000]
loss: 0.415694  [51264/60000]
loss: 0.434947  [57664/60000]
Test Error: 
 Accuracy: 85.2%, Avg loss: 0.394848 

Test Accuracy after Epoch 2: 85.23%
Test Loss after Epoch 2: 0.3948
Epoch 3
-------------------------------
loss: 0.220447  [   64/60000]
loss: 0.31

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
# Save the result to Google Sheets
worksheet.append_row([student_name,group_id, task_id, score])

print(f"Test completed! {student_name}, your score is {score}/{max_score}.")

Test completed! ELON MUSK, your score is 5/15.
